
# 1. Strands를 활용한 Agentic AI 이해 

**참조 사이트** : https://strandsagents.com/

## 1.1 사전 준비사항

- Python 3.8+ 설치
- AWS CLI 설정 및 Bedrock 모델 액세스 권한
- 기본적인 Python 프로그래밍 지식
- AWS 서비스에 대한 기본 이해


## 1.2 환경 설정

먼저 필요한 패키지들을 설치하고 AWS 자격 증명을 설정합니다.

In [ ]:
# 필요한 패키지 설치

pkg_list="strands-agents strands-agents-tools strands-agents-builder boto3 requests pprint"
! pip install --upgrade $pkg_list > /dev/null 2>&1
! pip list | egrep 'strands|boto3|requests' 

## 1.3 프로그램 초기 선언
- 초기 선언및 이 실습에 도움을 제공할 Helper 스크립트 정의

In [ ]:
# 필요한 라이브러리 임포트
import boto3
import json
import time
from pprint import pprint
from datetime import datetime
from typing import Dict, List, Any
from strands import Agent, tool 
from strands.models import BedrockModel

# --------------------------------------------------------------------
# 현재 Note BooK에 Role이 선언되어 있는 경우 AWS 자격 증명을 읽어 오기 위한 코드 
# --------------------------------------------------------------------
try:
    session = boto3.Session()
    credentials = session.get_credentials()
    print("AWS 자격 증명 확인 완료")
    print(f"현재 리전: {session.region_name}")
except Exception as e:
    print(f"AWS 자격 증명 오류: {e}")
    print("AWS CLI를 사용하여 자격 증명을 설정해주세요: aws configure")

# 현재 리전 확인
current_region = boto3.Session().region_name

# --------------------------------------------------------------------
# 직접 AWS 자격 증명 설정해야 하는경우에는 이곳을 선언해줘야함 
# --------------------------------------------------------------------
# aws_access_key_id = ''             # <-- IAM 사용자의 Access key 를 입력한다.
# aws_secret_access_key = ''         # <-- Secret Access key를 입력한다.
# region_name='us-east-1'            # <-- 리전을 선언한다.

# session = boto3.Session(
#     aws_access_key_id=aws_access_key_id,
#     aws_secret_access_key=aws_secret_access_key,
#     region_name=region_name
# )

# --------------------------------------------------------------------
# Bedrock Client 설정
# --------------------------------------------------------------------
bedrock = session.client(service_name='bedrock-runtime')
bedrock2 = session.client('bedrock')

# --------------------------------------------------------------------
# Bedrock Model ID 설정
# --------------------------------------------------------------------

# 이 실습에서 사용하는 기본 모델 선언 
base_model='amazon.nova-lite-v1:0'

# 현재 리전에서 사용할 Nova Model 정보 검색 
models = bedrock2.list_foundation_models()['modelSummaries']
nova_models = [model for model in models if base_model in model['modelId'].lower()]
print("사용한 Nova Model 정보 출력 :")
pprint(nova_models[0])


---

## 2. Strand Agent 


### 2.1 Stream & Non-stream 모드 선언 
- Stream은 Agent에 요청하고 응답하는 단계에서 Strem 형식으로 메세지를 확인하는 모드 이며
- Non-stream 모드는 Agent의 응답이 완성이 완료된 메세지만 받음.  

### 2.2 Stream mode Agent 선언후 Agent에 질의 


In [ ]:
# Stream 형식으로 모델 선언 
streaming_bedrock_model = BedrockModel(
    model_id=base_model,
    temperature=0.8,
    boto_session=session,
    streaming=True,  # Stream 형식으로 출력할지 선택 
)

# Agent 생성및 초기화 
stream_agent = Agent(model=streaming_bedrock_model)

# Stream 형식으로 Agent 실행하면서 질의 
stream_result=stream_agent("날씨에 관련된 재미있는 이야기를 해줘!")


### 2.2 Non-stream mode Agent 선언및 Agent에 질의 

In [ ]:
# Non-Stream 형식으로 모델 선언 
non_streaming_bedrock_model = BedrockModel(
    model_id=base_model,
    temperature=0.3,
    boto_session=session,
    streaming=False  # Non-Strem 형식으로 출력 
)

# Non-stream Agent 선언  
non_stream_agent = Agent(model=non_streaming_bedrock_model)

# Non-stream 형식으로 Agent 실행 
non_stream_result=non_stream_agent("토끼와 거북이 경주에 대한 재미있는 이야기를 해줘!")

### 2.3 callback_handler 를 사용한 메세지 출력 제한
- Agent가 실행중일 때 발생하는 메세지를 화면에 출력하지 않기 위한 방법

In [ ]:
print("[ Agent 실행 ]\n")
non_stream_agent = Agent(model=non_streaming_bedrock_model,callback_handler=None)
non_stream_result=non_stream_agent("스핑크스의 수수께끼 하나 알려줘!")
print("[ Agent 실행 완료 ]\n")

In [ ]:
# Agent 실행후에 메세지를 출력한다. 
print("\n[ Agent 실행결과 메세지 출력 ]\n")
print(non_stream_result)

### 2.4 System prompt를 함께 사용한 agent 선언및 실행
- 에이전트 생성시 전문분야나 역할등을 명시함

In [ ]:
# Agent 선언 
agent_system = Agent(
    model=streaming_bedrock_model,   # <-- 앞에서 선언한 bedrock model 선언 
    system_prompt="""
    당신은 친근한 AWS 전문가입니다!
    - 한국어로 쉽게 설명해주세요
    - 실용적인 예시를 들어주세요  
    - 이모지를 적절히 사용해주세요
    - 모든 답변에는 전문가적인 말투 대신에 아이들에게 이야기하는 말투로 해줘
    - 절대 거짓말을 하거나, 그럴듯사한 이야기를 하지말고 오직 정확한 답변만 해줘!
    - 모르면 모른다고 말해줘!
    """
)

# Agent 실행 
agent_system("AWS Lambda에서 VPC내에 RDS와 mysql로 연결하려면 어떻게 해야 할까요?")


### 2.5 [예문] 채팅 방식으로 카페에서 커피 주문하기 
- 매장에서 커피를 주문하듯이 반복적인 질문에 답을 합니다. 
- 입력창이 보이면 요청하는 내용대로 주문을 해보세요. 
- 주문을 완료하면 'quit' 또는 'exit' 를 입려해서 종료하세요!

In [ ]:
# 1. Stream 형식으로 모델 선언 
streaming_bedrock_model = BedrockModel(
    model_id=base_model,
    temperature=0.8,
    boto_session=session,
    streaming=True,  # Stream 형식으로 출력할지 선택 
)

# 2. System prompt 선언 
system_prompt = """
당신은 모든 것을 아름다운 시로 표현하는 낭만적인 바리스타입니다.

규칙:
1. 모든 답변을 3-4행의 시 형식으로 작성하세요
2. 커피와 관련된 감성적이고 시적인 표현을 사용하세요
3. 운율과 리듬감을 살려주세요
4. 주문 확인, 추천, 질문 답변 모두 시로 표현하세요

예시:
- 아메리카노 주문 시: "검은 밤하늘 담은 / 쓸쓸하면서도 깊은 / 에스프레소의 여운 / 지금 바로 준비하겠습니다"
- 추천 요청 시: "오늘 같은 날엔 / 부드러운 라떼 한 잔 / 우유 거품 위에 / 당신의 미소를 띄워보세요"
"""

# 3. 키보드 입력 
print("\n🎭 시인 바리스타가 당신을 기다립니다. 주문을 해주세요! (예: 아아 주세요) ....\n")
user_input = input("고객: ")
print("\n📜 시인 바리스타:")
print("─" * 60)

# 4. Agent 선언 
response = agent_system(user_input, system_prompt=system_prompt)

# 5. agent 실행 답변 
print(response)
print("─" * 60)

print("\n📜 시인 바리스타의 작별 인사:")
print("─" * 60)
print("오늘 만난 인연이")
print("향기로운 추억으로 남기를")
print("다음에 또 뵙겠습니다")
print("안녕히 가세요, 손님")
print("─" * 60)

---
## 3. Conversation History(대화기록)

- Agent 는 기본적으로 수명주기 동안 대화내용을 기록하고 있다.이 대화의 기록은 Agent가 수명주기를 다하거나, LLM에 입력되는 토큰수에 제한이 발생하는 이벤트가 발생이되면 오래된 대화내용을 제거하여 대화 내용을 관리하게 된다.
- 이 기본 기능은 대화 기록이 자동으로 유지되어 각 추론 시 모델에 전달되고, 도구 실행 컨텍스트로 사용되며, 컨텍스트 윈
도우 오버플로우를 방지하도록 관리된다.
- 기본 대화 기록 관리 외에도 Conversation Manager를 이용하여 대화내용을 관리할수 도 있다.

### 3.1 모델 초기화

In [ ]:
conversation_bedrock_model = BedrockModel(
    model_id=base_model,
    boto_session=session,
    streaming=True,  # Stream 형식으로 출력할지 선택 
)

### 3.2 Conversation History (대화기록)
- Agent가 기본적으로 대화 내용을 기억하는지 확인 


#### 3.2.1 Agent 선언및 실행
- Agent를 생성하면 기본적으로 단기 기억저장소가 자체적으로 생기고 초기화 된다

In [ ]:
# Stream mode로 agent 선언 및 Agent 초기화 
agent_memory = Agent(
    model=conversation_bedrock_model,
    system_prompt="""
    당신은 라면에 달인 입니다.
    - 한국어로 쉽게 설명해주세요
    - 실용적인 예시를 들어주세요  
    - 이모지를 적절히 사용해주세요
    - 텍스트형식으로 출력해주세요!
    """
)

# Agent에게 질의 
response=agent_memory("라면에 김치를 넣으면 어떨까요?")


#### 3.2.2 기억력 테스트를 위해 반복하여 질문해본다.

In [ ]:

response=agent_memory("라면에 파를 넣으면 어떨까요?")


#### 3.2.3 Agent에게 기억하고 있는 내용을 물어 본다.

In [ ]:
response=agent_memory("방금전에 내가 라면에 무엇을 넣어야 하는지 질문한 것들을 알려줘!")

#### 3.2.4 Agent가 기억하고 있는 모든 내용을 출력 한다. 
- 이때 agent.messages 를 사용한다. 이곳에는 모든 사용자와 어시스턴스의 메세지 뿐만 아니라 tool 사용과 결과에 대해서 기록이 이뤄진다.

In [ ]:
# Agent의 모든 기억 내용을 확인 하기 위해 messages 를 확인 한다. 
print(" ********  기억하고 있는 모든 대화 내용을 출력 *************\n")
for item in agent_memory.messages:
    print("\n")
    print(item)
print("\n")


#### 3.2.4 Agent가 기억하고 있는 내용중에 질문만 출력해본다.

In [ ]:
for text in agent_memory.messages :
    if text['role'] == 'user' :
        print(text)

### 3.3 Agent 초기화시 강제로 대화기록 삽입
- Agent 초기 설정시 대화기록을 삽입하도록 한다. 
- 이 기능을 이용하면 대화 기록을 장기 저장소에 저장해놓았다가 필요할때 가져와 사용할 수 있도록 할수 있다.

#### 3.3.1 Agent 초기화 한후 기존 대화 기록이 사라졌는지 확인 

In [ ]:
# Agent 선언및 초기화 
agent_memory = Agent(
    model=conversation_bedrock_model,
    system_prompt="""
    당신은 라면에 달인 입니다.
    - 한국어로 쉽게 설명해주세요
    - 실용적인 예시를 들어주세요  
    - 이모지를 적절히 사용해주세요
    - 텍스트형식으로 출력해주세요!
    """
)

# 기존 대화 기록 확인 
print("기존 대화 기록 : ")
print(agent_memory.messages)

#### 3.3.2 Agent를 초기화 하면서 대화 기록을 주입 (Memory 관리 기본)

In [ ]:
# 미리 대화기록을 작성한다.
pre_conv_history=[
    {'role': 'user', 'content': [{'text': '라면에 김치는 넣으면 어떨까요?'}]}, 
    {'role': 'assistant', 'content': [{'text': '# 라면에 김치 넣기 🍜 + 🥬\n\n라면에 김치를 넣는 것은 정말 훌륭한 조합이에요! 김치의 새콤함과 매콤함이 라면의 깊은 맛을 한층 더 끌어올려줍니다. 😋\n\n## 김치 라면의 장점 ✨\n- 김치의 발효된 맛이 라면 국물에 깊이를 더해줍니다\n- 아삭한 식감이 라면의 부드러운 면과 대비되어 식감이 풍부해져요\n- 김치에 있는 유산균이 라면의 건강 밸런스를 약간 개선해줍니다\n\n## 실용적인 팁 💡\n\n1. **신김치 vs 묵은지**\n   - 신김치: 아삭한 식감과 상큼한 맛을 원할 때 (국물 라면에 좋아요)\n   - 묵은지: 깊은 감칠맛을 원할 때 (볶음라면에 특히 좋습니다)\n\n2. **넣는 타이밍**\n   - 끓는 물에 면을 넣기 직전에 김치를 함께 넣으면 김치 맛이 국물에 잘 우러나요\n   - 볶음라면은 면을 거의 다 볶은 후 마지막에 김치를 넣어 살짝만 볶아주세요\n\n3. **추천 조합 예시** 🌟\n   - 신라면 + 잘 익은 배추김치 + 파 + 계란\n   - 진라면 + 묵은지 + 베이컨 조각 + 치즈 한 장\n   - 짜파게티 + 볶은 김치 (김치 볶음면의 정석!)\n\n김치 라면은 한국인의 소울푸드 조합이라고 할 수 있어요. 특히 해장이 필요할 때 더욱 빛을 발하는 조합입니다! 🔥'}]}   
]

# agent 생성시 대화 기록을 전달한다. 
agent_memory = Agent(
    model=conversation_bedrock_model,
    messages=pre_conv_history,
    system_prompt="""
    당신은 라면에 달인 입니다.
    - 한국어로 쉽게 설명해주세요
    - 실용적인 예시를 들어주세요  
    - 이모지를 적절히 사용해주세요
    - 텍스트형식으로 출력해주세요!
    """
)

# 대화 내용을 기억하고 있는지 확인 
response=agent_memory("방금전에 내가 라면에 무엇을 넣어야 한다고 했지?")

---
## 4. Tool(도구) 사용 
도구는 에이전트 기능을 확장하는 주요 메커니즘으로, 단순한 텍스트 생성을 넘어서 작업을 수행할 수 있게 해줍니다. 도구를 통해 에이전트는 외부 시스템과 상호작용하고, 데이터에 액세스하며, 환경을 조작할 수 있습니다.

- 참조: https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/example-tools-package/



### 4.1 도구 사용을 위한 모델 선언

In [ ]:

tooluse_bedrock_model = BedrockModel(
    model_id=base_model,
    boto_session=session,
    streaming=True,  # Stream 형식으로 출력할지 선택 
)

### 4.2 내장 Tool 구성 및 활용

#### 4.2.1 Tool 사용 승인 선언

일부 Tool 실행시 사용자 승인에 관련된 요청, 예를 들어 파일 저장관련 요청을 할경우 사용자의 입력 요청 ( Do you want to proceed with the file write? [y/*] ) 과 같은 처리시 사용자 요청을 자동처리 하기 위한 방법으로 운영체제 환경에 'BYPASS_TOOL_CONSENT' 값을 true로 선언한다.

In [ ]:
import os

os.environ["BYPASS_TOOL_CONSENT"] = "false"

#### 4.2.2 계산기 도구 사용

In [ ]:
from strands_tools import calculator

# Agent tool 선언 
tooluse_agent = Agent(
    model=tooluse_bedrock_model,
    tools=[calculator]
)

# Agent실행 
tooluse_agent("42 ^ 9 값을 무엇이야? 한국어로 대답해줘!")

print(f"\n실행한 도구 이름: {tooluse_agent.tool_names}")



#### 4.2.3 Agent가 모델이 도구를 사용하기 위한 대화내용 출력

In [ ]:
# Agent의 모든 기억 내용을 확인 하기 위해 messages 를 확인 한다. 
print(" ********  기억하고 있는 모든 대화 내용을 출력 *************\n")
for item in tooluse_agent.messages:
    print("\n")
    print(item)
print("\n")

#### 4.2.3 WEB URL 검색 도구 사용

In [ ]:
from strands_tools import http_request

# Agent tool 선언 
tooluse_agent = Agent(
    model=tooluse_bedrock_model,
    tools=[http_request],
    system_prompt="당신은 한국사람입니다. 한국어로 번역해서 답해주세요!"
)

# Agent실행 
response=tooluse_agent("다음 https://docs.aws.amazon.com/ko_kr/bedrock/latest/userguide/what-is-bedrock.html 사이트에 접속해서 정리 해줘!")



#### 4.2.3 여러 도구들 사용 - 파일 관리

In [ ]:
from strands_tools import file_read, file_write,current_time
import os

os.environ["BYPASS_TOOL_CONSENT"] = "true"

# Agent tool 선언 
tooluse_agent = Agent(
    model=tooluse_bedrock_model,
    tools=[file_read, file_write,current_time],
    system_prompt="당신은 한국사람입니다. 한국어로 번역해서 답해주세요!"
)

# tool사용을 요청하는 질의 
tooluse_agent("현재 디렉토리 아래에 tooluse_agent.dat 파일이 있는지 확인해주고, 파일이 없으면 파일을 생성하고, 파일에 현재 서울 시간을 기록해줘!")

print(f"\n실행한 도구 이름: {tooluse_agent.tool_names}")
os.environ["BYPASS_TOOL_CONSENT"] = "false"

#### 파일 생성 확인

In [ ]:
! ls -l tooluse_agent.dat

### 4.3 python function 방식의 tool 생성
- tool 을 function으로 설정함.

#### 4.3.1 날씨 검색 function 생성

In [ ]:
from strands import tool

@tool
def weather_forecast(city: str, days: int = 0) -> str:
    """도시의 날씨 예보 정보 가져옴.

    Args:
        city: 도시의 이름 (예: Seoul, Busan)
        days: 날씨 예보 일수 (0=오늘, 1=내일, 2=모레), 3일간만 제공 
    """
    import requests

    try:
        response = requests.get(f'http://wttr.in/{city}?format=j1')
        data = response.json()

        if days >= len(data['weather']):
            return f"{city}의 {days}일 후 예보 정보가 없습니다."

        weather_data = data['weather'][days]
        desc = weather_data['hourly'][4]['weatherDesc'][0]['value']
        max_temp = weather_data['maxtempC']
        min_temp = weather_data['mintempC']

        day_label = {0: "오늘", 1: "내일", 2: "모레"}.get(days, f"{days}일 후")

        return f"{city}의 {day_label} 날씨: {desc}, 최고 {max_temp}°C, 최저 {min_temp}°C"

    except Exception as e:
        return f"{city}의 날씨 정보를 가져올 수 없습니다: {str(e)}"


#### 4.3.2 생성된 tool 사용


In [ ]:
# Agent tool 선언 
tooluse_agent = Agent(
    model=tooluse_bedrock_model,
    tools=[weather_forecast]
)

# Agent실행 
tooluse_agent("서울의 오늘 날씨는?")  # 오늘을 포함한 3일간의 날씨만 제공함.

print(f"\n실행한 도구 이름: {tooluse_agent.tool_names}")


#### 4.3.3 난수발생 function들 생성

In [ ]:
# 소숫점 발생 난수 발생 tool
@tool
def random_float(min_value: float = 0.0, max_value: float = 1.0, precision: int = 2) -> str:
    """지정된 범위에서 실수 난수를 생성합니다.

    Args:
        min_value: 최솟값
        max_value: 최댓값
        precision: 소수점 자릿수
    """
    import random

    result = round(random.uniform(min_value, max_value), precision)
    return f"{min_value}~{max_value} 범위에서 생성된 실수 난수: {result}"

# 문자열 랜덤 발생 tool
@tool
def random_string(length: int = 8, include_numbers: bool = True, include_symbols: bool = False) -> str:
    """랜덤 문자열을 생성합니다.

    Args:
        length: 문자열 길이
        include_numbers: 숫자 포함 여부
        include_symbols: 특수문자 포함 여부
    """
    import random
    import string

    chars = string.ascii_letters
    if include_numbers:
        chars += string.digits
    if include_symbols:
        chars += "!@#$%^&*"

    result = ''.join(random.choice(chars) for _ in range(length))
    return f"길이 {length}의 랜덤 문자열: {result}"


#### 4.3.4 생성된 tool들 사용


In [ ]:
# Agent tool 선언 
tooluse_agent = Agent(
    model=tooluse_bedrock_model,
    tools=[weather_forecast,random_float,random_string]
)

# Agent실행 
tooluse_agent("1에서 10까지 난수를 만들어주고, 8문자의 숫자와 특수문자가 포함된 문자를 생성해줘!")
print(f"\n실행한 도구 이름: {tooluse_agent.tool_names}")


### 4.4 Direct tool 호출
- Agent에게 어떤 tool을 사용할지 직접 제안하는 방식으로 대화기록에 포함하는 방식과 그렇지 않는 방식으로 호출이 가능함.

#### 4.4.1 대화 기록에 포함하면서 직접 도구 호출
- 도구 실행의 결과를 대화기록에 포함하도록 실행함

In [ ]:
from strands_tools import calculator

tooluse_agent = Agent(
    model=tooluse_bedrock_model,
    tools=[calculator]
)

tooluse_agent.tool.calculator(expression="42 ^ 9")

print(f"\n실행한 도구 이름: {tooluse_agent.tool_names}")
print(f"\n대화기록: {tooluse_agent.messages}")
tooluse_agent("\n계산한 결과값이 뭐지?")


#### 4.5.2 대화 기록에 포함 하지 않으면서 직접 도구 호출
- 도구 실행의 결과를 대화기록에 포함하지 않도록 실행함

In [ ]:
from strands_tools import calculator

tooluse_agent = Agent(
    model=tooluse_bedrock_model,
    tools=[calculator]
)

tooluse_agent.tool.calculator(expression="42 ^ 9" , record_direct_tool_call=False)

print(f"\n실행한 도구 이름: {tooluse_agent.tool_names}")
print(f"\n대화기록: {tooluse_agent.messages}")
tooluse_agent("\n계산한 결과값이 뭐지?")


---
## 5. MCP tool 
- https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server 참조

### 5.1 MCP용 모델 선언

In [ ]:

mcp_bedrock_model = BedrockModel(
    model_id=base_model,
    boto_session=session
)

### 5.2 Streamable HTTP 방식의 MCP서버

#### 5.2.1 AWS 공식 문서 MCP 서버 연결  

In [ ]:
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient 

streamable_http_mcp_client = MCPClient(lambda: streamablehttp_client("https://knowledge-mcp.global.api.aws"))


#### 5.2.2 Agent 선언및 질의

In [ ]:

with streamable_http_mcp_client:
    # Get the tools from the MCP server
    tools = streamable_http_mcp_client.list_tools_sync()
    
    mcp_agent = Agent(model=mcp_bedrock_model,tools=tools)
    mcp_agent("Amazon cognito가 무엇이야? 그리고 문서 URL위치도 알려줘!")
    

## 6. 멀티 에이전트 - WorkFlow

---
### 6.1 WorkFlow

#### 6.1.1 순서적으로 여러 에이전트 실행 
```mermaid
graph LR
    A[조사 에이전트] --> B[분석 에이전트]
    B --> C[보고서 에이전트]
    
    style A fill:#1a1a1a,stroke:#00ff00,stroke-width:2px,color:#ffffff
    style B fill:#1a1a1a,stroke:#00ff00,stroke-width:2px,color:#ffffff
    style C fill:#1a1a1a,stroke:#00ff00,stroke-width:2px,color:#ffffff
```

In [ ]:

# 전문가 에이전트들 생성 
researcher_agent = Agent(system_prompt="당신은 연구 전문가입니다. 주요 정보를 찾아보세요.", model=base_model, callback_handler=None)
analyst_agent = Agent(system_prompt="연구 데이터를 분석하고 인사이트를 추출합니다.",model=base_model, callback_handler=None)
writer_agent = Agent(system_prompt="분석을 바탕으로 세련된 보고서를 작성합니다.",model=base_model,callback_handler=None)

topic = "짜장면"
# Step 1: Research
research_prompt=f"{topic} 에 대한 최신 개발 사항 연구"
print("\n [ 1. researcher agent 실행 중 ... ]\n ")
research_results = researcher_agent(research_prompt)

# Step 2: Analysis
analysis_prompt=f"이 연구 결과를 분석하십시오: {research_results}"
print("\n [ 2. analyst agent 실행 중 ... ]\n ")
analysis = analyst_agent(analysis_prompt)

# Step 3: Report writing
report_prompt=f"이 분석을 바탕으로 보고서를 작성합니다: {analysis}"
print("\n [ 3. writer agent 실행 중... ]\n")
final_report = writer_agent(report_prompt)

print("\n -- 최종 보고서 --\n")
print(final_report)